In [1]:
from __future__ import print_function
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.models import Model
from keras.datasets import cifar100
from keras.utils import to_categorical
import numpy as np
import os
import keras

Using TensorFlow backend.


In [2]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 100

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [3]:
# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [4]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


In [5]:
def resnet_v1(input_shape, depth, num_classes=100):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [6]:
def resnet_v2(input_shape, depth, num_classes=100):
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [7]:
import tensorflow as tf; print(tf.__version__)

1.14.0


In [8]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
model.summary()
print(model_type)

W1031 16:00:10.489040 140652491740992 deprecation_wrapper.py:119] From /home/medha/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1031 16:00:10.501352 140652491740992 deprecation_wrapper.py:119] From /home/medha/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1031 16:00:10.504150 140652491740992 deprecation_wrapper.py:119] From /home/medha/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4479: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1031 16:00:10.527386 140652491740992 deprecation_wrapper.py:119] From /home/medha/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_def

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [ ]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=True,
        # set each sample mean to 0
        samplewise_center=True,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=45,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.1,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=8,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/200
Learning rate:  0.001
1563/1563 [==============================] - 95s 61ms/step - loss: 2.9540 - acc: 0.3293 - val_loss: 3.9915 - val_acc: 0.2488

Epoch 00001: val_acc improved from -inf to 0.24880, saving model to /home/medha/Downloads/saved_models/cifar10_ResNet29v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
1563/1563 [==============================] - 96s 61ms/step - loss: 2.6826 - acc: 0.3829 - val_loss: 3.1317 - val_acc: 0.3281

Epoch 00002: val_acc improved from 0.24880 to 0.32810, saving model to /home/medha/Downloads/saved_models/cifar10_ResNet29v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
1563/1563 [==============================] - 93s 59ms/step - loss: 2.5652 - acc: 0.4119 - val_loss: 2.7844 - val_acc: 0.3827

Epoch 00003: val_acc improved from 0.32810 to 0.38270, saving model to /home/medha/Downloads/saved_models/cifar10_ResNet29v2_model.003.h5
Epoch 4/200
Learning rate:  0.001
1563/1563 [=============================

1563/1563 [==============================] - 100s 64ms/step - loss: 1.8237 - acc: 0.6092 - val_loss: 2.3968 - val_acc: 0.5025

Epoch 00034: val_acc did not improve from 0.53120
Epoch 35/200
Learning rate:  0.001
1563/1563 [==============================] - 100s 64ms/step - loss: 1.8188 - acc: 0.6126 - val_loss: 2.5874 - val_acc: 0.4725

Epoch 00035: val_acc did not improve from 0.53120
Epoch 36/200
Learning rate:  0.001
1563/1563 [==============================] - 100s 64ms/step - loss: 1.8151 - acc: 0.6129 - val_loss: 2.6554 - val_acc: 0.4761

Epoch 00036: val_acc did not improve from 0.53120
Epoch 37/200
Learning rate:  0.001
1563/1563 [==============================] - 102s 65ms/step - loss: 1.8122 - acc: 0.6155 - val_loss: 2.2838 - val_acc: 0.5437

Epoch 00037: val_acc improved from 0.53120 to 0.54370, saving model to /home/medha/Downloads/saved_models/cifar10_ResNet29v2_model.037.h5
Epoch 38/200
Learning rate:  0.001
1563/1563 [==============================] - 102s 65ms/step - lo

1563/1563 [==============================] - 100s 64ms/step - loss: 1.6574 - acc: 0.6582 - val_loss: 2.8352 - val_acc: 0.5109

Epoch 00071: val_acc did not improve from 0.57720
Epoch 72/200
Learning rate:  0.001
1563/1563 [==============================] - 101s 64ms/step - loss: 1.6475 - acc: 0.6630 - val_loss: 2.3140 - val_acc: 0.5338

Epoch 00072: val_acc did not improve from 0.57720
Epoch 73/200
Learning rate:  0.001
1563/1563 [==============================] - 101s 65ms/step - loss: 1.6494 - acc: 0.6607 - val_loss: 2.2542 - val_acc: 0.5620

Epoch 00073: val_acc did not improve from 0.57720
Epoch 74/200
Learning rate:  0.001
1563/1563 [==============================] - 102s 65ms/step - loss: 1.6428 - acc: 0.6630 - val_loss: 2.6362 - val_acc: 0.5058

Epoch 00074: val_acc did not improve from 0.57720
Epoch 75/200
Learning rate:  0.001
1563/1563 [==============================] - 102s 65ms/step - loss: 1.6435 - acc: 0.6636 - val_loss: 2.3910 - val_acc: 0.5342

Epoch 00075: val_acc did 

1563/1563 [==============================] - 94s 60ms/step - loss: 1.0136 - acc: 0.8028 - val_loss: 1.8378 - val_acc: 0.6438

Epoch 00106: val_acc did not improve from 0.65800
Epoch 107/200
Learning rate:  0.0001
1563/1563 [==============================] - 94s 60ms/step - loss: 1.0110 - acc: 0.8060 - val_loss: 1.8252 - val_acc: 0.6518

Epoch 00107: val_acc did not improve from 0.65800
Epoch 108/200
Learning rate:  0.0001
1563/1563 [==============================] - 95s 61ms/step - loss: 0.9975 - acc: 0.8098 - val_loss: 1.8552 - val_acc: 0.6488

Epoch 00108: val_acc did not improve from 0.65800
Epoch 109/200
Learning rate:  0.0001
1563/1563 [==============================] - 96s 61ms/step - loss: 0.9878 - acc: 0.8094 - val_loss: 1.8389 - val_acc: 0.6473

Epoch 00109: val_acc did not improve from 0.65800
Epoch 110/200
Learning rate:  0.0001
1563/1563 [==============================] - 98s 63ms/step - loss: 0.9874 - acc: 0.8100 - val_loss: 1.8612 - val_acc: 0.6455

Epoch 00110: val_acc d

1563/1563 [==============================] - 93s 59ms/step - loss: 0.8510 - acc: 0.8391 - val_loss: 1.8446 - val_acc: 0.6532

Epoch 00144: val_acc did not improve from 0.65910
Epoch 145/200
Learning rate:  1e-05
1563/1563 [==============================] - 94s 60ms/step - loss: 0.8561 - acc: 0.8385 - val_loss: 1.8280 - val_acc: 0.6550

Epoch 00145: val_acc did not improve from 0.65910
Epoch 146/200
Learning rate:  1e-05
1563/1563 [==============================] - 95s 61ms/step - loss: 0.8473 - acc: 0.8411 - val_loss: 1.8368 - val_acc: 0.6543

Epoch 00146: val_acc did not improve from 0.65910
Epoch 147/200
Learning rate:  1e-05
1563/1563 [==============================] - 97s 62ms/step - loss: 0.8430 - acc: 0.8427 - val_loss: 1.8243 - val_acc: 0.6551

Epoch 00147: val_acc did not improve from 0.65910
Epoch 148/200
Learning rate:  1e-05
1563/1563 [==============================] - 99s 64ms/step - loss: 0.8480 - acc: 0.8401 - val_loss: 1.8214 - val_acc: 0.6554

Epoch 00148: val_acc did n